### split train and val dataset 

In [34]:
import glob
import random
import os
data_dir = '/media/data3/lyx/Detection/data/train/images/'
files = sorted(glob.glob(data_dir+ "/*.*"))
file_ids = []
for each in files:
    file_ids.append(each.split('/')[-1][:-4])

data_num = len(file_ids)

total_ind = range(0, data_num)
val_ind = sorted(random.sample(range(0, data_num), data_num//5))

train_ind = [x for x in total_ind if x not in val_ind]

val_data = [file_ids[i] for i in val_ind]
train_data = [file_ids[i] for i in train_ind]
f = open('train_val.txt','w')
f.write(str(train_data))
f.write(str(val_data))
f.close()

label_dir = '/media/data3/lyx/Detection/data/train/labelXml/'
val_data_img = [data_dir+i+'.tif' for i in val_data]
train_data_img = [data_dir+i+'.tif' for i in train_data]
val_data_label = [label_dir+i+'.xml' for i in val_data]
train_data_label = [label_dir+i+'.xml' for i in train_data]


val_img_dst = '/media/data3/lyx/Detection/data_train_val/val/image/'
val_label_dst = '/media/data3/lyx/Detection/data_train_val/val/label/'
train_img_dst = '/media/data3/lyx/Detection/data_train_val/train/image/'
train_label_dst = '/media/data3/lyx/Detection/data_train_val/train/label/'
for path in (val_img_dst,val_label_dst,train_img_dst,train_label_dst):
    os.makedirs(path) 
val_data_img_dst = [val_img_dst+i+'.tif' for i in val_data]
train_data_img_dst = [train_img_dst+i+'.tif' for i in train_data]
val_data_label_dst = [val_label_dst+i+'.xml' for i in val_data]
train_data_label_dst = [train_label_dst+i+'.xml' for i in train_data]
import shutil


pair = zip([val_data_img,train_data_img,val_data_label,train_data_label],[val_data_img_dst,train_data_img_dst,val_data_label_dst,train_data_label_dst])
for src,dst in pair:
    for src,dst in zip(src,dst):
        shutil.copy(src, dst)

1000 4000


### Jittor version kld, gwd

In [33]:

import jittor as jt

from jittor.nn import bmm as bmm
from copy import deepcopy

from jittor import nn 

def xy_wh_r_2_xy_sigma(xywhr):
    """Convert oriented bounding box to 2-D Gaussian distribution.

    Args:
        xywhr (torch.Tensor): rbboxes with shape (N, 5).

    Returns:
        xy (torch.Tensor): center point of 2-D Gaussian distribution
            with shape (N, 2).
        sigma (torch.Tensor): covariance matrix of 2-D Gaussian distribution
            with shape (N, 2, 2).
    """
    _shape = xywhr.shape
    assert _shape[-1] == 5
    xy = xywhr[..., :2]
    wh = xywhr[..., 2:4].clamp(min_v=1e-7, max_v=1e7).reshape(-1, 2)
    r = xywhr[..., 4]
    cos_r = jt.cos(r)
    sin_r = jt.sin(r)
    R = jt.stack((cos_r, -sin_r, sin_r, cos_r), dim=-1).reshape(-1, 2, 2)
    S = 0.5 * jt.stack([jt.misc.diag(wh[i], diagonal=0) for i in range(wh.shape[0])]).reshape(-1, 2, 2)
    sigma = bmm(bmm(R,S*S),R.permute(0, 2,
                                            1)).reshape(_shape[:-1] + (2, 2))

    return xy, sigma


def postprocess(distance, fun='log1p', tau=1.0):

    if fun == 'log1p':
        distance = jt.log(distance+1)
    elif fun == 'sqrt':
        distance = jt.sqrt(distance.clamp(min_v=1e-7))
    elif fun == 'none':
        pass
    else:
        raise ValueError(f'Invalid non-linear function {fun}')

    if tau >= 1.0:
        return 1 - 1 / (tau + distance)
    else:
        return distance


def kld_loss_v0(pred, target, fun='log1p', tau=1.0, alpha=1.0, sqrt=True, weight=None, reduction='mean', avg_factor=None):

    xy_p, Sigma_p = pred
    xy_t, Sigma_t = target
    _shape = xy_p.shape

    xy_p = xy_p.reshape(-1, 2)
    xy_t = xy_t.reshape(-1, 2)
    Sigma_p = Sigma_p.reshape(-1, 2, 2)
    Sigma_t = Sigma_t.reshape(-1, 2, 2)
    Sigma_p_inv = jt.stack((Sigma_p[..., 1, 1], -Sigma_p[..., 0, 1],
                               -Sigma_p[..., 1, 0], Sigma_p[..., 0, 0]),
                              dim=-1).reshape(-1, 2, 2) 
    
    Sigma_p_inv = Sigma_p_inv / jt.linalg.det(Sigma_p).unsqueeze(-1).unsqueeze(-1)

    dxy = (xy_p - xy_t).unsqueeze(-1)
    xy_distance = 0.5 * bmm(bmm(dxy.permute(0, 2, 1),(Sigma_p_inv)), dxy).view(-1)

    whr_distance = 0.5 * bmm(Sigma_p_inv,Sigma_t)
    whr_distance = jt.stack([jt.misc.diag(whr_distance[i]) for i in range(whr_distance.shape[0])]).sum(dim=-1)


    Sigma_p_det_log = jt.linalg.det(Sigma_p).log()
    Sigma_t_det_log = jt.linalg.det(Sigma_t).log()

    whr_distance = whr_distance + 0.5 * (Sigma_p_det_log - Sigma_t_det_log)
    whr_distance = whr_distance - 1
    distance = (xy_distance / (alpha * alpha) + whr_distance)
    if sqrt:
        distance = distance.clamp(min_v=1e-7).sqrt()

    distance = distance.reshape(_shape[:-1])

    loss = postprocess(distance, fun=fun, tau=tau)
  
    if weight is not None:
        loss *= weight
    
    if avg_factor is None:
        avg_factor = loss.numel()
    
    if reduction=="sum":
        return loss.sum() 
    elif reduction == "mean":
        return loss.sum()/avg_factor
    return loss




def kld_loss(pred, target, fun='log1p', tau=1.0, weight=None, reduction='mean', avg_factor=None):
    xy_p, Sigma_p = pred
    xy_t, Sigma_t = target

    xy_p = xy_p.reshape(-1, 2)
    xy_t = xy_t.reshape(-1, 2)
    Sigma_p = Sigma_p.reshape(-1, 2, 2)
    Sigma_t = Sigma_t.reshape(-1, 2, 2)

    
    delta = (xy_p - xy_t).unsqueeze(-1)
    sigma_t_inv = jt.linalg.inv(Sigma_t)
    term1 = delta.transpose(-1,
                            -2).matmul(sigma_t_inv).matmul(delta).squeeze(-1)
    x_ = sigma_t_inv.matmul(Sigma_p)
    term2_ = jt.stack([jt.misc.diag(x_[i]) for i in range(x_.shape[0])]).sum(dim=-1).reshape(-1,1)
    term2 = term2_ + jt.log(jt.linalg.det(Sigma_t) / jt.linalg.det(Sigma_p)).reshape(-1, 1)
    
    dis = term1 + term2 - 2
    kl_dis = dis.clamp(min_v=1e-6)

    if fun == 'sqrt':
        loss = 1 - 1 / (tau + jt.sqrt(kl_dis))
    else:
        loss = 1 - 1 / (tau + jt.log(kl_dis+1))
    # return kl_loss
 
    if weight is not None:
        loss *= weight
    
    if avg_factor is None:
        avg_factor = loss.numel()
    
    if reduction=="sum":
        return loss.sum() 
    elif reduction == "mean":
        loss = loss.sum()/avg_factor
        # print('jt.grad(loss, Sigma_p)', jt.grad(loss, Sigma_p))
        return loss
    return loss


def gwd_loss_v0(pred, target, fun='log1p', tau=1.0, alpha=1.0, normalize=True, weight=None, reduction='mean', avg_factor=None):
    """Gaussian Wasserstein distance loss.

    """
    xy_p, Sigma_p = pred
    xy_t, Sigma_t = target

    xy_distance = ((xy_p - xy_t)*(xy_p - xy_t)).sum(dim=-1)
    
    whr_distance = jt.stack([jt.misc.diag(Sigma_p[i]) for i in range(Sigma_p.shape[0])]).sum(dim=-1)
    whr_distance = whr_distance + jt.stack([jt.misc.diag(Sigma_t[i]) for i in range(Sigma_t.shape[0])]).sum(dim=-1)
    x_ = bmm(Sigma_p,Sigma_t)
    _t_tr =jt.stack([jt.misc.diag(x_[i]) for i in range(x_.shape[0])]).sum(dim=-1)
    _t_det_sqrt = (jt.linalg.det(Sigma_p) * jt.linalg.det(Sigma_t)).clamp(1e-7).sqrt()
    whr_distance = whr_distance + (-2) * (
        (_t_tr + 2 * _t_det_sqrt).clamp(1e-7).sqrt())
    distance = (xy_distance + alpha * alpha * whr_distance).clamp(1e-7).sqrt()
    if normalize:
        scale = 2 * (
            _t_det_sqrt.clamp(1e-7).sqrt().clamp(1e-7).sqrt()).clamp(1e-7)
        distance = distance / scale

    
    loss = postprocess(distance, fun=fun, tau=tau)

    
    if weight is not None:
        loss *= weight
    
    if avg_factor is None:
        avg_factor = loss.numel()
    
    if reduction=="sum":
        return loss.sum() 
    elif reduction == "mean":
        return loss.sum()/avg_factor
    return loss

def gwd_loss(pred, target, fun='log1p', tau=2.0, normalize=True, weight=None, reduction='mean', avg_factor=None):
    """Gaussian Wasserstein distance loss.
    """
    xy_p, Sigma_p = pred
    xy_t, Sigma_t = target

    xy_distance = ((xy_p - xy_t)*(xy_p - xy_t)).sum(dim=-1)

    whr_distance = jt.stack([jt.misc.diag(Sigma_p[i]) for i in range(Sigma_p.shape[0])]).sum(dim=-1)
    whr_distance = whr_distance + jt.stack([jt.misc.diag(Sigma_t[i]) for i in range(Sigma_t.shape[0])]).sum(dim=-1)
    x_ = bmm(Sigma_p,Sigma_t)
    _t_tr =jt.stack([jt.misc.diag(x_[i]) for i in range(x_.shape[0])]).sum(dim=-1)
    _t_det_sqrt = (jt.linalg.det(Sigma_p) * jt.linalg.det(Sigma_t)).clamp(0).sqrt()
    whr_distance = whr_distance + (-2) * (
        (_t_tr + 2 * _t_det_sqrt).clamp(0).sqrt())
    distance = xy_distance + whr_distance
    gwd_dis = distance.clamp(min_v=1e-6)
    if fun == 'sqrt':
        loss = 1 - 1 / (tau + jt.sqrt(gwd_dis))
    elif fun == 'log1p':
        loss = 1 - 1 / (tau + jt.log(gwd_dis+1))
    else:
        scale = 2 * (_t_det_sqrt.sqrt().sqrt()).clamp(1e-7)
        loss = jt.log1p(jt.sqrt(gwd_dis) / scale)
    
    if weight is not None:
        loss *= weight
    
    if avg_factor is None:
        avg_factor = loss.numel()
    
    if reduction=="sum":
        return loss.sum() 
    elif reduction == "mean":
        return loss.sum()/avg_factor
    
    return loss

class GDLoss(nn.Module):

    BAG_GD_LOSS = {
        'gwd_v0': gwd_loss_v0,
        'kld_v0': kld_loss_v0,
        'gwd': gwd_loss,
        'kld': kld_loss,
    }
    BAG_PREP = {
        # 'xy_stddev_pearson': xy_stddev_pearson_2_xy_sigma,
        'xy_wh_r': xy_wh_r_2_xy_sigma
    }

    def __init__(self,
                 loss_type,
                 representation='xy_wh_r',
                 fun='log1p',
                 tau=1.0,
                 alpha=1.0,
                 reduction='mean',
                 loss_weight=1.0,
                 **kwargs):
        super(GDLoss, self).__init__()
        assert reduction in ['none', 'sum', 'mean']
        assert fun in ['log1p', 'none', 'sqrt']
        assert loss_type in self.BAG_GD_LOSS
        self.loss = self.BAG_GD_LOSS[loss_type]
        self.preprocess = self.BAG_PREP[representation]
        self.fun = fun
        self.tau = tau
        self.alpha = alpha
        self.reduction = reduction
        self.loss_weight = loss_weight
        self.kwargs = kwargs

    def execute(self,
                pred,
                target,
                weight=None,
                avg_factor=None,
                reduction_override=None,
                **kwargs):

        assert reduction_override in (None, 'none', 'mean', 'sum')
        reduction = (
            reduction_override if reduction_override else self.reduction)
        if (weight is not None) and (not jt.any(weight > 0)) and (
                reduction != 'none'):
            return (pred * weight).sum()
        if weight is not None and len(weight.shape) > 1:
            assert weight.shape == pred.shape
            weight = weight.mean(-1)
        _kwargs = deepcopy(self.kwargs)
        _kwargs.update(kwargs)
        pred_ = self.preprocess(pred)
    
        target_ = self.preprocess(target)
        loss = self.loss(
            pred_,
            target_,
            fun=self.fun,
            tau=self.tau,
            weight=weight,
            avg_factor=avg_factor,
            reduction=reduction,
            **_kwargs) * self.loss_weight
        # print('loss',loss)
        return loss

Loss = GDLoss('gwd_v0')

preds = jt.Var([[10.,15.,10.,20.,0.5],[10.,15.,10.,20.,0.5]])
target = jt.Var([[15.,15.,10.,5.,0.1],[15.,15.,10.,5.,0.1]])

Loss_gwd_v0 = GDLoss('gwd_v0')
Loss_gwd = GDLoss('gwd')
Loss_kld_v0 = GDLoss('kld_v0')
Loss_kld = GDLoss('kld')
L = (Loss_gwd_v0,Loss_gwd,Loss_kld_v0,Loss_kld )
for each in L:
    print(each(preds,target ))


jt.Var([0.3865626], dtype=float32)
jt.Var([0.81335044], dtype=float32)
jt.Var([0.43135446], dtype=float32)
jt.Var([0.7188362], dtype=float32)


### Pytorch version kld, gwd (From https://github.com/open-mmlab/mmrotate)

In [32]:
import torch
import torch.nn as nn

def xy_wh_r_2_xy_sigma(xywhr):
    """Convert oriented bounding box to 2-D Gaussian distribution.

    Args:
        xywhr (torch.Tensor): rbboxes with shape (N, 5).

    Returns:
        xy (torch.Tensor): center point of 2-D Gaussian distribution
            with shape (N, 2).
        sigma (torch.Tensor): covariance matrix of 2-D Gaussian distribution
            with shape (N, 2, 2).
    """
    _shape = xywhr.shape
    assert _shape[-1] == 5
    xy = xywhr[..., :2]
    wh = xywhr[..., 2:4].clamp(min=1e-7, max=1e7).reshape(-1, 2)
    r = xywhr[..., 4]
    cos_r = torch.cos(r)
    sin_r = torch.sin(r)
    R = torch.stack((cos_r, -sin_r, sin_r, cos_r), dim=-1).reshape(-1, 2, 2)
    S = 0.5 * torch.diag_embed(wh)

    sigma = R.bmm(S.square()).bmm(R.permute(0, 2,
                                            1)).reshape(_shape[:-1] + (2, 2))

    return xy, sigma

def postprocess(distance, fun='log1p', tau=1.0):
    """Convert distance to loss.

    Args:
        distance (torch.Tensor)
        fun (str, optional): The function applied to distance.
            Defaults to 'log1p'.
        tau (float, optional): Defaults to 1.0.

    Returns:
        loss (torch.Tensor)
    """
    if fun == 'log1p':
        distance = torch.log1p(distance)
    elif fun == 'sqrt':
        distance = torch.sqrt(distance.clamp(1e-7))
    elif fun == 'none':
        pass
    else:
        raise ValueError(f'Invalid non-linear function {fun}')

    if tau >= 1.0:
        return 1 - 1 / (tau + distance)
    else:
        return distance

def gwd_loss_v0(pred, target, fun='log1p', tau=1.0, alpha=1.0, normalize=True,weight=None, reduction='mean', avg_factor=None):

    xy_p, Sigma_p = pred
    xy_t, Sigma_t = target

    xy_distance = (xy_p - xy_t).square().sum(dim=-1)

    whr_distance = Sigma_p.diagonal(dim1=-2, dim2=-1).sum(dim=-1)
    whr_distance = whr_distance + Sigma_t.diagonal(
        dim1=-2, dim2=-1).sum(dim=-1)

    _t_tr = (Sigma_p.bmm(Sigma_t)).diagonal(dim1=-2, dim2=-1).sum(dim=-1)
    _t_det_sqrt = (Sigma_p.det() * Sigma_t.det()).clamp(1e-7).sqrt()
    whr_distance = whr_distance + (-2) * (
        (_t_tr + 2 * _t_det_sqrt).clamp(1e-7).sqrt())

    distance = (xy_distance + alpha * alpha * whr_distance).clamp(1e-7).sqrt()

    if normalize:
        scale = 2 * (
            _t_det_sqrt.clamp(1e-7).sqrt().clamp(1e-7).sqrt()).clamp(1e-7)
        distance = distance / scale
    
    loss = postprocess(distance, fun=fun, tau=tau)
    if weight is not None:
        loss *= weight
    
    if avg_factor is None:
        avg_factor = loss.numel()
    
    if reduction=="sum":
        return loss.sum() 
    elif reduction == "mean":
        return loss.sum()/avg_factor
    
    return loss


def kld_loss_v0(pred, target, fun='log1p', tau=1.0, alpha=1.0, sqrt=True,weight=None, reduction='mean', avg_factor=None):
    """Kullback-Leibler Divergence loss.

    Args:
        pred (torch.Tensor): Predicted bboxes.
        target (torch.Tensor): Corresponding gt bboxes.
        fun (str): The function applied to distance. Defaults to 'log1p'.
        tau (float): Defaults to 1.0.
        alpha (float): Defaults to 1.0.
        sqrt (bool): Whether to sqrt the distance. Defaults to True.

    Returns:
        loss (torch.Tensor)
    """
    xy_p, Sigma_p = pred
    xy_t, Sigma_t = target

    _shape = xy_p.shape

    xy_p = xy_p.reshape(-1, 2)
    xy_t = xy_t.reshape(-1, 2)
    Sigma_p = Sigma_p.reshape(-1, 2, 2)
    Sigma_t = Sigma_t.reshape(-1, 2, 2)

    Sigma_p_inv = torch.stack((Sigma_p[..., 1, 1], -Sigma_p[..., 0, 1],
                               -Sigma_p[..., 1, 0], Sigma_p[..., 0, 0]),
                              dim=-1).reshape(-1, 2, 2)
    Sigma_p_inv = Sigma_p_inv / Sigma_p.det().unsqueeze(-1).unsqueeze(-1)

    dxy = (xy_p - xy_t).unsqueeze(-1)
    xy_distance = 0.5 * dxy.permute(0, 2, 1).bmm(Sigma_p_inv).bmm(dxy).view(-1)

    whr_distance = 0.5 * Sigma_p_inv.bmm(Sigma_t).diagonal(
        dim1=-2, dim2=-1).sum(dim=-1)

    Sigma_p_det_log = Sigma_p.det().log()
    Sigma_t_det_log = Sigma_t.det().log()
    whr_distance = whr_distance + 0.5 * (Sigma_p_det_log - Sigma_t_det_log)
    whr_distance = whr_distance - 1
    distance = (xy_distance / (alpha * alpha) + whr_distance)
    if sqrt:
        distance = distance.clamp(1e-7).sqrt()

    distance = distance.reshape(_shape[:-1])

    loss = postprocess(distance, fun=fun, tau=tau)
    if weight is not None:
        loss *= weight
    
    if avg_factor is None:
        avg_factor = loss.numel()
    
    if reduction=="sum":
        return loss.sum() 
    elif reduction == "mean":
        return loss.sum()/avg_factor
    
    return loss
    



def gwd_loss(pred, target, fun='log1p', tau=2.0,weight=None, reduction='mean', avg_factor=None):
    """Gaussian Wasserstein distance loss.

    Args:
        pred (torch.Tensor): Predicted bboxes.
        target (torch.Tensor): Corresponding gt bboxes.
        fun (str): The function applied to distance. Defaults to 'log1p'.
        tau (float): Defaults to 1.0.

    Returns:
        loss (torch.Tensor)
    """
    mu_p, sigma_p = pred
    mu_t, sigma_t = target

    xy_distance = (mu_p - mu_t).square().sum(dim=-1)

    whr_distance = sigma_p.diagonal(dim1=-2, dim2=-1).sum(dim=-1)
    whr_distance = whr_distance + sigma_t.diagonal(
        dim1=-2, dim2=-1).sum(dim=-1)

    _t_tr = (sigma_p.bmm(sigma_t)).diagonal(dim1=-2, dim2=-1).sum(dim=-1)
    _t_det_sqrt = (sigma_p.det() * sigma_t.det()).clamp(0).sqrt()
    whr_distance += (-2) * (_t_tr + 2 * _t_det_sqrt).clamp(0).sqrt()

    dis = xy_distance + whr_distance
    gwd_dis = dis.clamp(min=1e-6)

    if fun == 'sqrt':
        loss = 1 - 1 / (tau + torch.sqrt(gwd_dis))
    elif fun == 'log1p':
        loss = 1 - 1 / (tau + torch.log1p(gwd_dis))
    else:
        scale = 2 * (_t_det_sqrt.sqrt().sqrt()).clamp(1e-7)
        loss = torch.log1p(torch.sqrt(gwd_dis) / scale)
    
    if weight is not None:
        loss *= weight
    
    if avg_factor is None:
        avg_factor = loss.numel()
    
    if reduction=="sum":
        return loss.sum() 
    elif reduction == "mean":
        return loss.sum()/avg_factor

    return loss



def kld_loss(pred, target, fun='log1p', tau=1.0,weight=None, reduction='mean', avg_factor=None):
    """Kullback-Leibler Divergence loss.

    Args:
        pred (torch.Tensor): Predicted bboxes.
        target (torch.Tensor): Corresponding gt bboxes.
        fun (str): The function applied to distance. Defaults to 'log1p'.
        tau (float): Defaults to 1.0.

    Returns:
        loss (torch.Tensor)
    """
    mu_p, sigma_p = pred
    mu_t, sigma_t = target

    mu_p = mu_p.reshape(-1, 2)
    mu_t = mu_t.reshape(-1, 2)
    sigma_p = sigma_p.reshape(-1, 2, 2)
    sigma_t = sigma_t.reshape(-1, 2, 2)

    delta = (mu_p - mu_t).unsqueeze(-1)
    sigma_t_inv = torch.inverse(sigma_t)
    term1 = delta.transpose(-1,
                            -2).matmul(sigma_t_inv).matmul(delta).squeeze(-1)
    term2 = torch.diagonal(
        sigma_t_inv.matmul(sigma_p),
        dim1=-2, dim2=-1).sum(dim=-1, keepdim=True) + \
        torch.log(torch.det(sigma_t) / torch.det(sigma_p)).reshape(-1, 1)
    dis = term1 + term2 - 2
    kl_dis = dis.clamp(min=1e-6)

    if fun == 'sqrt':
        loss = 1 - 1 / (tau + torch.sqrt(kl_dis))
    else:
        loss = 1 - 1 / (tau + torch.log1p(kl_dis))
    
    if weight is not None:
        loss *= weight
    
    if avg_factor is None:
        avg_factor = loss.numel()
    
    if reduction=="sum":
        return loss.sum() 
    elif reduction == "mean":
        return loss.sum()/avg_factor

    return loss



class GDLoss(nn.Module):
    """Gaussian based loss.

    Args:
        loss_type (str):  Type of loss.
        representation (str, optional): Coordinate System.
        fun (str, optional): The function applied to distance.
            Defaults to 'log1p'.
        tau (float, optional): Defaults to 1.0.
        alpha (float, optional): Defaults to 1.0.
        reduction (str, optional): The reduction method of the
            loss. Defaults to 'mean'.
        loss_weight (float, optional): The weight of loss. Defaults to 1.0.

    Returns:
        loss (torch.Tensor)
    """
    BAG_GD_LOSS = {
        'gwd': gwd_loss,
        'kld': kld_loss,
        'gwd_v0': gwd_loss_v0,
        'kld_v0': kld_loss_v0,
    }
    BAG_PREP = {
        'xy_wh_r': xy_wh_r_2_xy_sigma
    }

    def __init__(self,
                 loss_type,
                 representation='xy_wh_r',
                 fun='log1p',
                 tau=1.0,
                 reduction='mean',
                 loss_weight=1.0,
                 **kwargs):
        super(GDLoss, self).__init__()
        assert reduction in ['none', 'sum', 'mean']
        assert fun in ['log1p', 'none', 'sqrt']
        assert loss_type in self.BAG_GD_LOSS
        self.loss = self.BAG_GD_LOSS[loss_type]
        self.preprocess = self.BAG_PREP[representation]
        self.fun = fun
        self.tau = tau
        self.reduction = reduction
        self.loss_weight = loss_weight
        self.kwargs = kwargs

    def forward(self,
                pred,
                target,
                weight=None,
                avg_factor=None,
                reduction_override=None,
                **kwargs):
        """Forward function.

        Args:
            pred (torch.Tensor): Predicted convexes.
            target (torch.Tensor): Corresponding gt convexes.
            weight (torch.Tensor, optional): The weight of loss for each
                prediction. Defaults to None.
            avg_factor (int, optional): Average factor that is used to average
                the loss. Defaults to None.
            reduction_override (str, optional): The reduction method used to
               override the original reduction method of the loss.
               Defaults to None.
        """
        assert reduction_override in (None, 'none', 'mean', 'sum')
        reduction = (
            reduction_override if reduction_override else self.reduction)
        if (weight is not None) and (not torch.any(weight > 0)) and (
                reduction != 'none'):
            return (pred * weight).sum()
        if weight is not None and weight.dim() > 1:
            assert weight.shape == pred.shape
            weight = weight.mean(-1)
        _kwargs = deepcopy(self.kwargs)
        _kwargs.update(kwargs)

        pred = self.preprocess(pred)
        target = self.preprocess(target)

        return self.loss(
            pred,
            target,
            fun=self.fun,
            tau=self.tau,
            weight=weight,
            avg_factor=avg_factor,
            reduction=reduction,
            **_kwargs) * self.loss_weight


preds = torch.Tensor([[10.,15.,10.,20.,0.5],[10.,15.,10.,20.,0.5]])
target = torch.Tensor([[15.,15.,10.,5.,0.1],[15.,15.,10.,5.,0.1]])
Loss_gwd_v0 = GDLoss('gwd_v0')
Loss_gwd = GDLoss('gwd')
Loss_kld_v0 = GDLoss('kld_v0')
Loss_kld = GDLoss('kld')
L = (Loss_gwd_v0,Loss_gwd,Loss_kld_v0,Loss_kld )
for each in L:
    print(each(preds,target ))


tensor(0.3866)
tensor(0.8134)
tensor(0.4314)
tensor(0.7188)


In [11]:
import torch
a = torch.Tensor([[[ 2.1162066e+00, -2.2864749e+00],
  [-5.5145769e+00,  1.3518972e+01]],

 [[ 1.0000001e+00, -7.0014124e-08],
  [ 9.3700834e-09,  1.0000000e+00]]])

torch.diagonal( a,
        dim1=-2, dim2=-1).sum(dim=-1, keepdim=True)

tensor([[15.6352],
        [ 2.0000]])